In [ ]:
from kafka import KafkaConsumer
import json
import pandas as pd

import findspark
findspark.init()
from pyspark.sql.types import StructType, StructField, LongType, IntegerType, StringType



import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row

from pyspark.sql.functions import col

scala_version = '2.12'
spark_version = '3.5.3'

packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.9.0'
]

spark = SparkSession.builder.master('local').appName('streaming_data').config('spark.jars.packages', ",".join(packages)).getOrCreate()
# Tạo KafkaConsumer để nhận dữ liệu từ một Kafka topic
def create_consumer(topic):
    """
    Tạo KafkaConsumer để nhận dữ liệu từ một Kafka topic.
    """
    consumer = KafkaConsumer(
        topic,
        bootstrap_servers=['localhost:9092'],  # Kafka broker
        value_deserializer=lambda x: json.loads(x.decode('utf-8')),  # Chuyển đổi dữ liệu JSON
        auto_offset_reset='latest'  # Đảm bảo chỉ lấy dữ liệu mới

    )
    return consumer

# Hàm tiêu thụ dữ liệu từ Kafka và xử lý
def consume_data(consumer):
    """
    Tiêu thụ dữ liệu từ Kafka và xử lý.
    """
    print("đang chờ, chưa có dữ liệu")
    
    for message in consumer:
        print(f"Received message: {message.value}")
        
        # Thêm vào danh sách các dữ liệu đã nhận
        message_data = message.value
        
        # Chuyển dictionary thành Row để tạo Spark DataFrame
        row = Row(**message_data)
        
        # Tạo DataFrame từ Row
        df = spark.createDataFrame([row])

        # Hiển thị DataFrame
        df.show()
if __name__ == "__main__":
    topic = "resultData"  # Thay đổi theo tên topic mà bạn muốn tiêu th
    consumer = create_consumer(topic)
    print("Đang chờ dữ liệu từ Kafka...")
    consume_data(consumer)


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from time import sleep
from IPython.display import clear_output
import findspark
findspark.init()

# Tạo SparkSession
scala_version = '2.12'
spark_version = '3.5.3'

packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.9.0'
]

spark = SparkSession.builder.master('local').appName('streaming_data').config('spark.jars.packages', ",".join(packages)).getOrCreate()

# Kafka settings
KAFKA_BROKER = "localhost:9092"
TOPIC_NAME = "resultData"

# Định nghĩa schema của dữ liệu nhận được từ Kafka
schema_value = StructType([
    StructField("processed_comment", StringType(), True),
    StructField("sentiment", IntegerType(), True)
])

# Đọc dữ liệu từ Kafka
df_stream = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", KAFKA_BROKER) \
    .option("subscribe", TOPIC_NAME) \
    .load()

# Chuyển đổi dữ liệu từ Kafka (byte) thành DataFrame và áp dụng schema
df_json = df_stream.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), schema_value).alias("data")) \
    .select("data.*")

# Hàm hiển thị dữ liệu trong DataFrame sau mỗi batch
def show_batch(df, epoch_id):
    # Sử dụng show() để hiển thị 10 dòng đầu tiên của DataFrame
    df.show(10, truncate=False)

# Cập nhật và hiển thị dữ liệu vào console (hoặc có thể gửi đến các hệ thống khác)
query = df_json.writeStream \
    .outputMode("append") \
    .trigger(processingTime='5 seconds') \
    .foreachBatch(show_batch) \
    .start()


# Chờ để kết thúc query
query.awaitTermination()


+-----------------+---------+
|processed_comment|sentiment|
+-----------------+---------+
+-----------------+---------+

+----------------------------------------------------------------------------------------+---------+
|processed_comment                                                                       |sentiment|
+----------------------------------------------------------------------------------------+---------+
|không bt đẹp giao hàng nhanh shiper thân_thiện giá rẻ chất_lượng shop uy_tín hình_ảnh xu|0        |
|bluetoth không dây kết_nối bluetoth tiện lại sử_dụng                                    |0        |
|được được màu xinh nghe không hư_hỏng âm chát không ấm nghe nhạc không hay chất ổn mua  |0        |
|dây tai dây kết_nối bluetoth tiện lại                                                   |0        |
|tai đã hàng tai khui bật chút ngắt tiền giá                                             |0        |
|được nhưng mà pin quá sạc                                             